In [1]:
import numpy as np

In [2]:
import scipy

In [4]:
import scipy.io
data=scipy.io.loadmat('train_.mat')
X=data['X']
Y=data['y']

In [ ]:
Y

In [122]:
Y = Y.flatten()

In [123]:
Y

array([1, 9, 2, ..., 1, 6, 9], dtype=uint8)

## Reshaping the matrices 

In [5]:
l=[]
for i in range(73257):
    j=X[:,:,:,i].reshape(1024,3)
    l.append([j])

X_2= np.concatenate(l)

In [4]:
X_2.shape

(73257, 1024, 3)

In [6]:
X_3 = X_2.reshape(73257,3072)

In [6]:
X_3.shape

(73257, 3072)

Loading Testing Data

In [426]:

data=scipy.io.loadmat('test_.mat')
X_test=data['X']
Y_test=data['y']
Y_test =Y_test.flatten()

l=[]
for i in range(26032):
    j=X_test[:,:,:,i].reshape(1024,3)
    l.append([j])

X_test = np.concatenate(l)
X_test = X_test.reshape(26032,3072)

## Creating the Neural Network

In [394]:

def  loss_func(soft_max,true_value) :

   m = -np.log(soft_max[true_value-1])
   
   return m
    # m = list(zip(predicted_value,true_value))
    # L = 0
    # for i,j in m:
    #     g = j*(np.log(i)) + (1-j)*(1-np.log(1-i))
    #     L += g


    # L = (1/len(predicted_value))*L

    # return L

def sigmoid(x):

    return 1/(1 + np.exp(-x))

def diff_sigm(x):

    return (x)*(1-x)

def compute_softmax(x):
   
   l = np.vectorize(np.exp)(x)

   s = np.sum(l)

   l = l/s

   return l 
   
def compute_gradient(p,x,y):
   
   weights = p.weights
   weights = np.flip(weights,0)
   output = p.predict(x)
   softmax = output[2]
   output_for_layers = output[1]
   # output_for_layers = np.flip(output_for_layers,axis=0)
   gradient_wrt_output = []
   gradient_wrt_weights = []
   gradients = []
   output_with_weights = []
   
   softmax[y - 1] = softmax[y - 1] - 1
   gradient_wrt_output = softmax
   
  
   tuple_output_for_layers = [(np.array(output_for_layers[i]),np.array(output_for_layers[i+1])) for i in range(len(output_for_layers)-1)]
   tuple_output_for_layers.reverse()

   
   i = 0
   gradient_layer = gradient_wrt_output

   while i != len(weights) :
      gradients = []
    
      m = [gradient_layer,np.vectorize(diff_sigm)(tuple_output_for_layers[i][1]),np.append(tuple_output_for_layers[i][0],1)]

      for t in range(len(gradient_layer)):
         k = (m[0][t]*m[1][t])*m[2]
         gradients.append(k)
      gradients = np.array(gradients,dtype ='float64')   
      
      gradient_wrt_weights.append(gradients)
      gradient_layer_2 = []
      j = (np.array(weights[i])).transpose()
      p = np.multiply(gradient_layer,m[1])
      for k in j :

         n = np.sum(np.multiply(p,k))
         gradient_layer_2.append(n)
      gradient_layer_2.pop()
      # l = []
      # for g in gradient_layer_2:

      #    f = g.astype('float64')
      #    l.append(f)

      # gradient_layer_2 = np.array(l)

      # gradient_layer_2 = np.array(gradient_layer_2)

      gradient_layer = gradient_layer_2

      i+=1


   gradient_wrt_weights = np.array(gradient_wrt_weights)
   gradient_wrt_weights = np.flip(gradient_wrt_weights,axis = 0)

   return gradient_wrt_weights


   # for j in range(len(gradient_wrt_output)):

   # for i in range(len(weights)):
      
   #    d = list(zip(weights[i],output_for_layers[i]))
   #    output_with_weights.append(d)

   # output_with_weights = np.flip(output_with_weights,axis = 0)  


   # for i in output_with_weights:
   #    layer_gradient = []
   #    # for j in range(len(i)-1):



In [395]:
class Neural_Network :

    def __init__(self,num_of_layers=3,input_size=1000,output_size=10,number_neurons=[0],classes=[]):
        
        if len(number_neurons)!= num_of_layers :
            raise ValueError('length of hidden_neurons should be equal to the number of hidden layers')
        
        
        self.num_of_layers = num_of_layers
        self.input_size = input_size
        self.output_size = output_size
        self.number_neurons = number_neurons
        self.classes = classes
    
    
    def weight_initialization(self,technique='Xavier'):
        
        if technique == 'Xavier':
            number_neurons = self.number_neurons
            
            weights = []
            for i in range(1,len(number_neurons)):
               
                if i != len(number_neurons)-1:
                    layer_weight=[]
                    var = 2 / (number_neurons[i-1] + (number_neurons[i+1])) # 
                    sigma = np.sqrt(var)
                else:
                    layer_weight=[]
                    var = 2 / (number_neurons[i-1]) # 
                    sigma = np.sqrt(var)

            
                
                for b in range(number_neurons[i]):
                    l = np.random.normal(0,sigma,number_neurons[i-1]+1)
                    # l = l.astype('float64')
                    layer_weight.append(l)
                # layer_weight = np.array(layer_weight,dtype = 'float64')
                weights.append(layer_weight)
            
            weights=np.array(weights)
            self.weights = weights
            return weights 
            
    def predict(self,x):
        y = self.weights
        i = 0
        L = x
        output_array = []
        output_array.append(x)
        while len(L)!= 10:
            M=[]
            for j in y[i]:
                k_1 = np.dot(j[:-1], L)
                k = k_1 + j[-1]
                k = 1/(1 + np.exp(-k))                               # Sigmoid activation function 
                M.append(k) 
                

            output_array.append(M)
            L = np.array(M)
            i+=1


            
        L = compute_softmax(L)

       
        
        
        m = np.max(L)

        class_label = np.where(L == m)

        class_label =  class_label[0][0] + 1

        output_array = np.array(output_array)
        predictions = (class_label,output_array,L)
    
        return predictions

    def test(self,test_data, kind = 'Macro'):
         
        Classes_dict = {}

        for i in range(1,11):
            Classes_dict[i] = [0,0,0]

        Input,Output = (test_data[0],test_data[1])


        for j in range(len(Input)):

            d = self.predict(Input[j])
            d = d[0]

            if d == Output[j]:
                Classes_dict[d][0]+=1
            else:
                Classes_dict[Output[j]][2]+=1
                Classes_dict[d][1]+=1

        if kind == 'Macro':

                l = []

                for i in Classes_dict:
                    if Classes_dict[i][0] != 0:
                        k = Classes_dict[i][0]/(Classes_dict[i][0]+Classes_dict[i][1])
                        l.append(k)
                    else:
                        l.append(0)    

                l = np.array(l)

                precision = np.sum(l)/10

                l = []
                for i in Classes_dict:
                    if Classes_dict[i][0] != 0:
                        k = Classes_dict[i][0]/(Classes_dict[i][0]+Classes_dict[i][2])
                        l.append(k)
                    else :
                        l.append(0)

                l = np.array(l)

                recall = np.sum(l)/10


                f_1_score = 2*precision*recall/(precision + recall)
                
                m = 0
                for i in Classes_dict:
                    m+=Classes_dict[i][0]

                accuracy = m/len(Input)



                return (f_1_score,precision,recall,accuracy)

        elif kind == 'Micro':

                l = []
                t_p = 0
                f_p = 0

                for i in Classes_dict:
                   t_p += Classes_dict[i][0]
                   f_p += Classes_dict[i][1]
                   
                precision = t_p/(t_p + f_p)
                

                l = []
                t_p  = 0
                f_p = 0

                for i in Classes_dict:
                   t_p += Classes_dict[i][0]
                   f_n += Classes_dict[i][2]
                   
                recall = t_p/(t_p + f_n) 


                f_1_score = 2*precision*recall/(precision + recall)


                m = 0
                for i in Classes_dict:
                    m+=Classes_dict[i][0]

                accuracy = m/len(Input)



                return (f_1_score,precision,recall,accuracy)

    # def compute_gradient(self,x,y):
    #     gradient = []
    #     rev_array = np.flip(x)
    #     for i in x:

    #         for j in i:





In [432]:
class optimizers : 

    def __init__(self) :
        self.weights = []
        self.index = 0
        self.epoch = 0


    # def SGD(self,neural_network,training_data,batch_size,epochs,learning_rate):

    #     number_of_batches = len(training_data[0]) // batch_size
    #     for f in range(epochs):
    #         gradient = np.zeros(neural_network.weights.shape, dtype = 'O' )
    #         for k in range(number_of_batches):
    #             i,j = (training_data[0][(k-1)*batch_size:k*batch_size],training_data[1][(k-1)*batch_size:k*batch_size])
    #             weight_gradient = np.zeros(neural_network.weights.shape,dtype = 'O' )
    #             while len(i) != 0:

    #                 Input = i[-1]
    #                 Output = j[-1]
    #                 i = np.delete(i,-1,0)
    #                 j = np.delete(j,-1)
    #                 d =  compute_gradient(neural_network,Input,Output)
    #                 # d = d.astype('float64')
    #                 weight_gradient += d
    #             weight_gradient = (1/(batch_size))*weight_gradient

    #             gradient += weight_gradient


            


    #         gradient = (1/number_of_batches)*gradient

    #         neural_network.weights -= learning_rate*gradient


    #     return None
    
    def SGD(self,neural_network,training_data,epochs,learning_rate):

        l = [(training_data[0][i],training_data[1][i]) for i in range(len(training_data[0])-1000)]
        l = np.array(l)
        i = 0
        validation_set = (training_data[0][-1000:],training_data[1][-1000:])
        while i!= epochs:
            np.random.shuffle(l)
            self.epoch +=1

            for j in l:
                self.index +=1
                d = compute_gradient(neural_network,j[0],j[1])
                neural_network.weights-= learning_rate*d
                if self.index % 5000 == 0:
                    stopping_criteria = neural_network.test(validation_set)[0]
                    self.weights.append(stopping_criteria)
           
                    if stopping_criteria >= .9 :
                        return None
                    
            
            i+=1

        return None


    def RMSPROP(self,neural_network,training_data,epochs,learning_rate,rho,epsilon):

        l = [(training_data[0][i],training_data[1][i]) for i in range(len(training_data[0])-1000)]
        l = np.array(l)
        Lambda = learning_rate
        i = 0
        v_h = 0
        d_2 = 0
        validation_set = (training_data[0][-1000:],training_data[1][-1000:])
        while i!= epochs:
            np.random.shuffle(l)
            
            for j in l:
                self.index +=1
                
                d = compute_gradient(neural_network,j[0],j[1])
                d = np.square(d)
                d_2 = d_2*rho + d*(1-rho)
                learning_rate = Lambda/(d_2 + epsilon)
                neural_network.weights-= learning_rate*v_h
                if self.index % 5000 == 0:
                    stopping_criteria = neural_network.test(validation_set)[0]
                    self.weights.append(stopping_criteria)
                    if stopping_criteria >= .9:
                        return None
                    
            
            i+=1

        return None


**Implementing the neural network for Stochastic Gradient Descent and RMSProp. You may test the network by running these codes . Implementing the optimizers took about 250 minutes for each RMSProp and SGD**

In [ ]:
m = Neural_Network(3,3072,10,[3072,1500,10])
weights = m.weight_initialization()
#f = m.predict(X_3[0])
#gr = compute_gradient(m,X_3[0], Y[0])

sd = optimizers()

## Implementing the SGD optimizer

In [ ]:
sd.SGD(m,(X_3,Y),5,.1)



##  Testing the weights obtained by SGD optimizer on test data

In [ ]:
z = m.test((X_test,Y_test))


In [434]:
z

(0.03120555596167358,
 0.018421793359413543,
 0.10196154089123825,
 0.07767363245236632)

## f_1_score = 0.03120555596167358
## precision = 0.018421793359413543
## recall = 0.10196154089123825
## accuracy = 0.07767363245236632

## Implementing the RMSProp optimizer

In [ ]:
m = Neural_Network(3,3072,10,[3072,1500,10])
weights = m.weight_initialization()
sd = optimizers()
sd.RMSPROP(m,(X_3,Y),5,.001,.9,1e-08)

In [435]:
m.test((X_test,Y_test))

/var/folders/85/tt6k5_h12qsgwhr9kvl_f4hr0000gn/T/ipykernel_8602/3576540119.py:78: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  output_array = np.array(output_array)
/var/folders/85/tt6k5_h12qsgwhr9kvl_f4hr0000gn/T/ipykernel_8602/3576540119.py:57: RuntimeWarning: overflow encountered in exp
  k = 1/(1 + np.exp(-k))                               # Sigmoid activation function


(0.05343012075465881,
 0.03568537371432808,
 0.10627687992054129,
 0.08163030116779349)